<a href="https://colab.research.google.com/github/imabari/ImabariScraping/blob/master/mynumbar_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pandas

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.1)


In [2]:
import csv
import datetime

import pandas as pd

In [3]:
def df_conv(df, col_name, col_1="人口", col_2="交付枚数"):

    df.set_axis(col_name, axis=1, inplace=True)

    population_date = df.iloc[-1][col_1].strftime("%Y/%m/%d")
    delivery_date = df.iloc[-1][col_2].strftime("%Y/%m/%d")

    df["人口算出基準日"] = population_date
    df["交付枚数算出基準日"] = delivery_date
    df.insert(0, "算出基準日", delivery_date)

    return df.iloc[:-1]

In [4]:
df = pd.read_excel(
    "https://www.soumu.go.jp/main_content/000703058.xlsx", sheet_name=1
).sort_index(ascending=False)

df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
1864,時点,NaN,2019-01-01 00:00:00,2020-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1863,沖縄県,八重山郡与那国町,1716,323,0.188228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1862,沖縄県,八重山郡竹富町,4343,908,0.209072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861,沖縄県,宮古郡多良間村,1172,102,0.0870307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1860,沖縄県,島尻郡八重瀬町,31338,4126,0.131661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,特別区,9486618,2258503,0.238073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,全国,127443563,23246822,0.182409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,区分,人口,交付枚数,人口に対する交付枚数率,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 団体区分別,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfg = df.groupby((df["Unnamed: 0"] == "時点").cumsum())

# 市区町村別

In [6]:
df1 = df_conv(
    dfg.get_group(1)
    .sort_index()
    .dropna(axis=1, how="all")
    .dropna(how="all")
    .iloc[2:]
    .reset_index(drop=True),
    ["都道府県名", "市区町村名", "総数（人口）", "交付枚数", "人口に対する交付枚数率"],
    "総数（人口）",
    "交付枚数",
)

In [7]:
df1["人口に対する交付枚数率"] = df1["人口に対する交付枚数率"].astype(float).round(3) * 100

df1["市区町村名"] = df1["市区町村名"].replace(r"\s", "", regex=True)
df1["市区町村名"] = df1["市区町村名"].mask(df1["都道府県名"] + df1["市区町村名"] == "兵庫県篠山市", "丹波篠山市")
df1["市区町村名"] = df1["市区町村名"].mask(df1["都道府県名"] + df1["市区町村名"] == "高知県高岡郡梼原町", "高岡郡檮原町")
df1["市区町村名"] = df1["市区町村名"].mask(df1["都道府県名"] + df1["市区町村名"] == "福岡県糟屋郡須惠町", "糟屋郡須恵町")

if pd.Timestamp(df1.iloc[0]["算出基準日"]) < datetime.date(2018, 10, 1):
    df1["市区町村名"] = df1["市区町村名"].mask(
        df1["都道府県名"] + df1["市区町村名"] == "福岡県那珂川市", "筑紫郡那珂川町"
    )
else:
    df1["市区町村名"] = df1["市区町村名"].mask(
        df1["都道府県名"] + df1["市区町村名"] == "福岡県筑紫郡那珂川町", "那珂川市"
    )

In [8]:
df_code = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vSseDxB5f3nS-YQ1NOkuFKZ7rTNfPLHqTKaSag-qaK25EWLcSL0klbFBZm1b6JDKGtHTk6iMUxsXpxt/pub?gid=0&single=true&output=csv",
    dtype={"団体コード": int, "都道府県名": str, "郡名": str, "市区町村名": str},
)

df_code["市区町村名"] = df_code["郡名"].fillna("") + df_code["市区町村名"]
df_code.drop("郡名", axis=1, inplace=True)

In [9]:
df1 = pd.merge(df1, df_code, on=["都道府県名", "市区町村名"], how="left")
df1["団体コード"] = df1["団体コード"].astype("Int64")

In [10]:
df1.to_csv(
    "all_localgovs.csv",
    index=False,
    quoting=csv.QUOTE_MINIMAL,
    float_format="%.1f",
    encoding="utf_8_sig",
)

In [11]:
df1

,算出基準日,都道府県名,市区町村名,総数（人口）,交付枚数,人口に対する交付枚数率,人口算出基準日,交付枚数算出基準日,団体コード
0,2020/08/01,北海道,札幌市,1955457,312994,16.0,2019/01/01,2020/08/01,11002
1,2020/08/01,北海道,函館市,258948,38507,14.9,2019/01/01,2020/08/01,12025
2,2020/08/01,北海道,小樽市,116529,19129,16.4,2019/01/01,2020/08/01,12033
3,2020/08/01,北海道,旭川市,337392,49404,14.6,2019/01/01,2020/08/01,12041
4,2020/08/01,北海道,室蘭市,84405,15371,18.2,2019/01/01,2020/08/01,12050
...,...,...,...,...,...,...,...,...,...
1736,2020/08/01,沖縄県,島尻郡久米島町,7873,744,9.5,2019/01/01,2020/08/01,473618
1737,2020/08/01,沖縄県,島尻郡八重瀬町,31338,4126,13.2,2019/01/01,2020/08/01,473626
1738,2020/08/01,沖縄県,宮古郡多良間村,1172,102,8.7,2019/01/01,2020/08/01,473758
1739,2020/08/01,沖縄県,八重山郡竹富町,4343,908,20.9,2019/01/01,2020/08/01,473812


# 男女・年齢別

In [12]:
df2 = df_conv(
    dfg.get_group(2)
    .sort_index()
    .dropna(axis=1, how="all")
    .dropna(how="all")
    .iloc[3:]
    .reset_index(drop=True),
    [
        "年齢",
        "人口(男)",
        "人口(女)",
        "人口(計)",
        "交付件数(男)",
        "交付件数(女)",
        "交付件数(計)",
        "交付率(男)",
        "交付率(女)",
        "交付率(計)",
        "全体に対する交付件数割合(男)",
        "全体に対する交付件数割合(女)",
        "全体に対する交付件数割合(計)",
    ],
    "人口(男)",
    "交付件数(男)",
)

In [13]:
df2["交付率(男)"] = df2["交付率(男)"].astype(float).round(3) * 100
df2["交付率(女)"] = df2["交付率(女)"].astype(float).round(3) * 100
df2["交付率(計)"] = df2["交付率(計)"].astype(float).round(3) * 100
df2["全体に対する交付件数割合(男)"] = df2["全体に対する交付件数割合(男)"].astype(float).round(3) * 100
df2["全体に対する交付件数割合(女)"] = df2["全体に対する交付件数割合(女)"].astype(float).round(3) * 100
df2["全体に対する交付件数割合(計)"] = df2["全体に対する交付件数割合(計)"].astype(float).round(3) * 100

In [14]:
df2.to_csv(
    "demographics.csv",
    index=False,
    quoting=csv.QUOTE_MINIMAL,
    float_format="%.1f",
    encoding="utf_8_sig",
)

In [15]:
df2

,算出基準日,年齢,人口(男),人口(女),人口(計),交付件数(男),交付件数(女),交付件数(計),交付率(男),交付率(女),交付率(計),全体に対する交付件数割合(男),全体に対する交付件数割合(女),全体に対する交付件数割合(計),人口算出基準日,交付枚数算出基準日
0,2020/08/01,全体,62174142,65269421,127443563,12195103,11051719,23246822,19.6,16.9,18.2,100.0,100.0,100.0,2019/01/01,2020/08/01
1,2020/08/01,0～4歳,2521289,2396069,4917358,136029,129525,265554,5.4,5.4,5.4,1.1,1.2,1.1,2019/01/01,2020/08/01
2,2020/08/01,5～9,2732810,2597758,5330568,193365,188262,381627,7.1,7.2,7.2,1.6,1.7,1.6,2019/01/01,2020/08/01
3,2020/08/01,10～14,2823910,2686588,5510498,186461,190412,376873,6.6,7.1,6.8,1.5,1.7,1.6,2019/01/01,2020/08/01
4,2020/08/01,15～19,3041288,2892530,5933818,308275,310822,619097,10.1,10.7,10.4,2.5,2.8,2.7,2019/01/01,2020/08/01
5,2020/08/01,20～24,3296677,3124747,6421424,520671,534875,1055546,15.8,17.1,16.4,4.3,4.8,4.5,2019/01/01,2020/08/01
6,2020/08/01,25～29,3310343,3124346,6434689,624164,588142,1212306,18.9,18.8,18.8,5.1,5.3,5.2,2019/01/01,2020/08/01
7,2020/08/01,30～34,3658968,3493161,7152129,700428,620999,1321427,19.1,17.8,18.5,5.7,5.6,5.7,2019/01/01,2020/08/01
8,2020/08/01,35～39,4002907,3856691,7859598,736439,617267,1353706,18.4,16.0,17.2,6.0,5.6,5.8,2019/01/01,2020/08/01
9,2020/08/01,40～44,4656293,4495779,9152072,804585,642463,1447048,17.3,14.3,15.8,6.6,5.8,6.2,2019/01/01,2020/08/01


# 都道府県一覧

In [16]:
df3 = df_conv(
    dfg.get_group(3)
    .sort_index()
    .dropna(axis=1, how="all")
    .dropna(how="all")
    .iloc[2:]
    .reset_index(drop=True),
    ["都道府県名", "総数（人口）", "交付枚数", "人口に対する交付枚数率"],
    "総数（人口）",
)

df3["人口に対する交付枚数率"] = df3["人口に対する交付枚数率"].astype(float).round(3) * 100

In [17]:
df3.to_csv(
    "all_prefectures.csv",
    index=False,
    quoting=csv.QUOTE_MINIMAL,
    float_format="%.1f",
    encoding="utf_8_sig",
)

In [18]:
df3

,算出基準日,都道府県名,総数（人口）,交付枚数,人口に対する交付枚数率,人口算出基準日,交付枚数算出基準日
0,2020/08/01,北海道,5304413,840125,15.8,2019/01/01,2020/08/01
1,2020/08/01,青森県,1292709,208252,16.1,2019/01/01,2020/08/01
2,2020/08/01,岩手県,1250142,203738,16.3,2019/01/01,2020/08/01
3,2020/08/01,宮城県,2303098,405129,17.6,2019/01/01,2020/08/01
4,2020/08/01,秋田県,1000223,154453,15.4,2019/01/01,2020/08/01
5,2020/08/01,山形県,1095383,153131,14.0,2019/01/01,2020/08/01
6,2020/08/01,福島県,1901053,298994,15.7,2019/01/01,2020/08/01
7,2020/08/01,茨城県,2936184,522595,17.8,2019/01/01,2020/08/01
8,2020/08/01,栃木県,1976121,339538,17.2,2019/01/01,2020/08/01
9,2020/08/01,群馬県,1981202,293690,14.8,2019/01/01,2020/08/01


# 団体区分別

In [19]:
df4 = df_conv(
    dfg.get_group(6)
    .sort_index()
    .dropna(axis=1, how="all")
    .dropna(how="all")
    .iloc[2:]
    .reset_index(drop=True),
    ["区分", "人口", "交付枚数", "人口に対する交付枚数率"],
)

df4["人口に対する交付枚数率"] = df4["人口に対する交付枚数率"].astype(float).round(3) * 100

In [20]:
df4 = df4.round({"人口に対する交付枚数率": 1})

In [21]:
df4.to_csv(
    "summary_by_types.csv",
    index=False,
    quoting=csv.QUOTE_MINIMAL,
    float_format="%.1f",
    encoding="utf_8_sig",
)

In [22]:
df4

,算出基準日,区分,人口,交付枚数,人口に対する交付枚数率,人口算出基準日,交付枚数算出基準日
0,2020/08/01,全国,127443563,23246822,18.2,2019/01/01,2020/08/01
1,2020/08/01,特別区,9486618,2258503,23.8,2019/01/01,2020/08/01
2,2020/08/01,政令指定都市,27488569,5340870,19.4,2019/01/01,2020/08/01
3,2020/08/01,市（政令指定都市を除く）,79573258,13962375,17.5,2019/01/01,2020/08/01
4,2020/08/01,町村,10895118,1685074,15.5,2019/01/01,2020/08/01
